# A Simple Framework for Contrastive Learning of Visual Representations


* URL: https://arxiv.org/abs/2002.05709
* authors: Ting Chen, Simon Kornblith, Mohammad Norouzi, Geoffrey Hinton
* Submitted on 13 Feb 2020 (v1)

## 1. どんなもの？
SimCLR, which stands for a simple framework for contrastive learning of visual representationsの提案。

## 2. 先行研究と比べるとどこが凄い？  
SimCLRは先行研究に比べてシンプルでありながら、特別なアーキテクチャやメモリーバンクを必要とせずに高い性能を出すことができる

## 3. 技術や手法のキモはどこ？

### flow
1. data augmentation  
   データに拡張を施すことによって同じサンプルからの異なるデータを生成し、それをpositive pairとする。  
   ramdom cropping -> resize back to original size -> random color distortions -> random Gaussaian blur(色々実験して決めた。)(contrastive learningではデータの拡張が非常に大事になっている。拡張の度合いを高めると性能が良くなることを示した。)
2. a neural network base encoder  
   ここではResNetを選択した。average pooling layerを通す。
   $$
   h_{i} = f(\widetilde{x}_{i}) = \text{ResNet}(\widetilde{x}_{i}), \text{which } h_{i} \in \text{ is the output after the average pooling layer.}
   $$
3. a small neural network projection head  
   contrastive lossを適用できるような次元にマップする関数。$h_{i}$にcontrasitive lossを適用するよりもこの層を通した方が良かった。下流タスクに適用するときはこの層は除く。
   $$
   z_{i} = g(h_{i}) = W^{(2)}\sigma{W^{(1)}h_{i}}, \text{which } \sigma \text{ is a ReLU nonlinearity.}
   $$
4. contrasitive lossを適用。
   
ランダムにサンプルしたmini-batch$N$に対してデータ拡張を適用するとデータは$2N$になる。ここでネガティブサンプルは$2N-1$を使用する。(MultipleNegativesRankingLossと似た感じよね)

### NT-Xent, which stands for the normalized temperature-scaled cross entropy loss

$$
\mathcal{l}_{i, j} = -\log{\frac{\exp(\text{sim}(z_{i}, z_{j}/\tau))}{\sum_{k=1}^{2N}{\mathbf{1}_{[k \not ={i}]}\exp(\text{sim}(z_{i}, z_{j}/\tau))}}}
$$
where, $\text{sim}(\mathbf{u}, \mathbf{v}) = \mathbf{u}^\top\mathbf{v}/\| \mathbf{u}\| \| \mathbf{v}\|$, $\mathbf{u}, \mathbf{v}$はnormalizedされたもの。$\tau$はtemperature parameter。
すべてのpositive pairに対して対称的に(i->j, j->i)計算される。

### large batchで訓練
large batchではSGDは安定しないのでoptimizerとしてLARS optimizerを使用。また、ResNetのBatch Normalizationは全てのデバイスで適用した。(それぞれのデバイスで適用するとrepresentationを改良することなく正解率を上げるためにデータリークを利用することができるから) <-どういうことだろ。

## 4. どうやって有効だと検証した？
Metricsがいまいち分からない。Linear Evaluation?  
(https://blog.recruit.co.jp/data/articles/ssl_vision_01/#fn:10)
> 学習された後のEncoderの重みを凍結した上で、最終層に一層だけ線形層をつけて学習を行い分類精度を評価すること。学習されたEncoderから得られるEmbedding空間でそれぞれの画像のクラスが線形分類しやすいようにクラスタに分かれていると分類精度も高くなるため、教師情報を与えずに学習したEmbeddingがどの程度後流のタスクに有用かを測る手法としてよく用いられる。  

恐ろしいほどの数のデータセットやモデルを用いて検証している。

## 5. 議論はある？

### 6. 次に読むべき論文は？

### 7. 実装
https://github.com/google-research/simclr